In [99]:
import os

In [67]:
os.chdir("Exe-Project")

FileNotFoundError: [Errno 2] No such file or directory: 'Exe-Project'

In [108]:
!pwd


/Users/vaishnavishankardevadig/Downloads/Exe-Project


In [109]:
from dataclasses import dataclass
from pathlib import Path

In [110]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [111]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

#Configuration.py

In [112]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        print(self.config)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [113]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.commons2 import getsizeb


In [114]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download with this information: \n{headers}")
        else:
            logger.info(f"File already exists. Size: {getsizeb(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts zip file into data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [115]:
#Pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-15 21:22:58,037: INFO: common: yaml file: config/config.yaml loaded successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/vdevadig/Exe-Project/raw/main/Skin_Data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
[2024-02-15 21:22:58,038: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-15 21:22:58,039: INFO: common: Created directory at: artifacts]
[2024-02-15 21:22:58,039: INFO: common: Created directory at: artifacts/data_ingestion]
[2024-02-15 21:22:59,331: INFO: 2989780564: artifacts/data_ingestion/data.zip download with this information: 
Connection: close
Content-Length: 23671640
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "351c99e327f3ff0ab93b11397a53abc86d7ac1e67b2d5496e970e8c605a2bba1"
Strict-Transport-Security: max-

In [64]:
print(os.getcwd()) 

/Users/vaishnavishankardevadig/Downloads/Exe-Project


In [65]:
print(f"Resolved config path: {CONFIG_FILE_PATH.resolve()}")
print(f"Resolved params path: {PARAMS_FILE_PATH.resolve()}")


Resolved config path: /Users/vaishnavishankardevadig/Downloads/Exe-Project/config/config.yaml
Resolved params path: /Users/vaishnavishankardevadig/Downloads/Exe-Project/params.yaml
